### Cài đặt thư viện

In [1]:
!pip3 install -q unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.0/353.0 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 28.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 77.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.5/283.5 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 13.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.8/899.8 MB 1.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.4/170.4 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.5 

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"  # Sử dụng cả 2 GPU

In [3]:
%%capture
# We're installing the latest Torch, Triton, OpenAI's Triton kernels, Transformers and Unsloth!
!pip install --upgrade -qqq uv
try: import numpy; install_numpy = f"numpy=={numpy.__version__}"
except: install_numpy = "numpy"
!uv pip install -qqq \
    "torch>=2.8.0" "triton>=3.4.0" {install_numpy} \
    "unsloth_zoo[base] @ git+https://github.com/unslothai/unsloth-zoo" \
    "unsloth[base] @ git+https://github.com/unslothai/unsloth" \
    torchvision bitsandbytes \
    git+https://github.com/huggingface/transformers \
    git+https://github.com/triton-lang/triton.git@05b2c186c1b6c9a08375389d5efe9cb4c401c075#subdirectory=python/triton_kernels

### Load model gốc

In [4]:
from unsloth import FastLanguageModel
import torch

# PATCH: Bypass check chat template của Unsloth
import unsloth.tokenizer_utils as tokenizer_utils

original_fix_chat_template = tokenizer_utils.fix_chat_template

def patched_fix_chat_template(tokenizer):
    """Bypass strict check"""
    try:
        return original_fix_chat_template(tokenizer)
    except RuntimeError as e:
        if "add_generation_prompt" in str(e):
            # Force add template if missing
            tokenizer.chat_template = "{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}"
            return tokenizer.chat_template
        raise e

tokenizer_utils.fix_chat_template = patched_fix_chat_template

# Bây giờ load bình thường
max_seq_length = 1024
dtype = None
model_name = "AITeamVN/Vi-Qwen2-3B-RAG"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    dtype=dtype,
    max_seq_length=max_seq_length,
    load_in_4bit=True,
)

print("✅ Model loaded successfully!")
print(f"Chat template: {tokenizer.chat_template[:100]}...")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-11-24 03:50:07.992111: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763956208.180896      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763956208.234427      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.3: Fast Qwen2 patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

✅ Model loaded successfully!
Chat template: {% for message in messages %}{{'<|im_start|>' + message['role'] + '
' + message['content'] + '<|im_e...


### Add LORA

In [5]:
# model = FastLanguageModel.get_peft_model(
#     model,
#     r = 8, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
#     target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
#                       "gate_proj", "up_proj", "down_proj",],
#     lora_alpha = 16,
#     lora_dropout = 0, # Supports any, but = 0 is optimized
#     bias = "none",    # Supports any, but = "none" is optimized
#     # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
#     use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
#     random_state = 3407,
#     use_rslora = False,  # We support rank stabilized LoRA
#     loftq_config = None, # And LoftQ
# )

### Load data để fine tune model

In [6]:
import pandas as pd
import json
from datasets import Dataset
from unsloth import FastLanguageModel
from trl import SFTConfig, SFTTrainer
import torch

# ===========================
# 1. Load và chuẩn bị dữ liệu
# ===========================

# def parse_label(label_str):
#     """Parse label string thành dict với aspect categories và sentiments"""
#     aspects = {}
#     # Format: {CATEGORY#Sentiment};{CATEGORY#Sentiment};...
#     parts = label_str.strip().split(';')
#     for part in parts:
#         if '#' in part:
#             category, sentiment = part.strip('{}').split('#')
#             aspects[category] = sentiment
#     return aspects

# def create_absa_prompt(comment, aspects_dict):
#     """Tạo prompt training cho ABSA task"""
#     # Format aspects thành JSON
#     aspects_list = list(aspects_dict.keys())
    
#     # System message
#     system_msg = """Bạn là chuyên gia phân tích đánh giá sản phẩm điện thoại.
# Nhiệm vụ: Trích xuất các khía cạnh (aspect) và phân loại cảm xúc (sentiment) từ đánh giá.

# Các khía cạnh hợp lệ: SCREEN, CAMERA, FEATURES, BATTERY, PERFORMANCE, STORAGE, DESIGN, PRICE, GENERAL, SER&ACC
# Các cảm xúc hợp lệ: Positive, Negative, Neutral"""
    
#     # User message - câu đánh giá
#     user_msg = f'Phân tích đánh giá: "{comment}"'
    
#     # Assistant response - kết quả dạng JSON
#     assistant_response = json.dumps(aspects_dict, ensure_ascii=False)
    
#     return [
#         {"role": "system", "content": system_msg},
#         {"role": "user", "content": user_msg},
#         {"role": "assistant", "content": assistant_response}
#     ]

def parse_label(label_str):
    """Parse label string thành dict với aspect categories và sentiments"""
    aspects = {}
    # Format: {CATEGORY#Sentiment};{CATEGORY#Sentiment};...
    parts = label_str.strip().split(';')
    for part in parts:
        if '#' in part:
            category, sentiment = part.strip('{}').split('#')
            aspects[category] = sentiment
    return aspects

def create_absa_prompt(comment, aspects_dict):
    """Tạo prompt training cho ABSA task"""
    # System message với format rõ ràng
    system_msg = """Bạn là chuyên gia phân tích đánh giá sản phẩm điện thoại.
Nhiệm vụ: Trích xuất các khía cạnh (aspect) và phân loại cảm xúc (sentiment) từ đánh giá.

Các khía cạnh hợp lệ: SCREEN, CAMERA, FEATURES, BATTERY, PERFORMANCE, STORAGE, DESIGN, PRICE, GENERAL, SER&ACC
Các cảm xúc hợp lệ: Positive, Negative, Neutral

CHỈ trả về JSON object theo format:
{
  "ASPECT1": "Sentiment1",
  "ASPECT2": "Sentiment2"
}"""
    
    # User message - câu đánh giá
    user_msg = f'Phân tích đánh giá: "{comment}"'
    
    # Assistant response - format JSON đẹp với indent
    assistant_response = json.dumps(aspects_dict, ensure_ascii=False, indent=2)
    
    return [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": user_msg},
        {"role": "assistant", "content": assistant_response}
    ]

# Load dữ liệu
train_df = pd.read_csv("/kaggle/input/uit-visfd/Train.csv")
val_df = pd.read_csv("/kaggle/input/uit-visfd/Dev.csv")

print(f"Train samples: {len(train_df)}")
print(f"Val samples: {len(val_df)}")

# Parse labels và tạo conversations
train_data = []
for _, row in train_df.iterrows():
    aspects = parse_label(row['label'])
    if aspects:  # Chỉ lấy samples có label hợp lệ
        messages = create_absa_prompt(row['comment'], aspects)
        train_data.append({"messages": messages})

val_data = []
for _, row in val_df.iterrows():
    aspects = parse_label(row['label'])
    if aspects:
        messages = create_absa_prompt(row['comment'], aspects)
        val_data.append({"messages": messages})

print(f"Processed train samples: {len(train_data)}")
print(f"Processed val samples: {len(val_data)}")

# Tạo HuggingFace Datasets
train_dataset = Dataset.from_list(train_data)
val_dataset = Dataset.from_list(val_data)

# ===========================
# 2. Load Model với Unsloth
# ===========================

max_seq_length = 2048
dtype = None
model_name = "unsloth/Qwen2.5-3B-Instruct"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    dtype=dtype,
    max_seq_length=max_seq_length,
    load_in_4bit=True,
)

print("✅ Model loaded successfully!")

# ===========================
# 3. Add LoRA adapters
# ===========================

model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # LoRA rank
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

print("✅ LoRA adapters added!")

# ===========================
# 4. Chuẩn bị tokenizer và formatting
# ===========================

def formatting_prompts_func(examples):
    """Format messages thành text để train"""
    texts = []
    for messages in examples["messages"]:
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False
        )
        texts.append(text)
    return {"text": texts}

# Apply formatting
train_dataset = train_dataset.map(
    formatting_prompts_func,
    batched=True,
    remove_columns=["messages"]
)

val_dataset = val_dataset.map(
    formatting_prompts_func,
    batched=True,
    remove_columns=["messages"]
)

print("✅ Datasets formatted!")

Train samples: 7786
Val samples: 1112
Processed train samples: 7671
Processed val samples: 1094
==((====))==  Unsloth 2025.11.3: Fast Qwen2 patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.36G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

Unsloth 2025.11.3 patched 36 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


✅ LoRA adapters added!


Map:   0%|          | 0/7671 [00:00<?, ? examples/s]

Map:   0%|          | 0/1094 [00:00<?, ? examples/s]

✅ Datasets formatted!


### Train model

In [7]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    args=SFTConfig(
        # Multi-GPU training (tự động detect)
        per_device_train_batch_size=8,  # Giảm xuống vì có 2 GPU
        per_device_eval_batch_size=8,
        gradient_accumulation_steps=4,
        warmup_steps=50,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="cosine",
        seed=3407,
        output_dir="outputs",
        eval_strategy="steps",
        eval_steps=100,
        save_strategy="steps",
        save_steps=100,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        
        # Tối ưu cho Multi-GPU
        ddp_find_unused_parameters=False,
        dataloader_num_workers=4,
        dataloader_pin_memory=True,
        remove_unused_columns=False,
        
        report_to="none",
    ),
)


Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/7671 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/1094 [00:00<?, ? examples/s]

In [8]:
# Train
trainer_stats = trainer.train()

print("✅ Training completed!")
print(f"Training loss: {trainer_stats.training_loss:.4f}")

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,671 | Num Epochs = 3 | Total steps = 720
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 4 x 1) = 32
 "-____-"     Trainable parameters = 29,933,568 of 3,115,872,256 (0.96% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
100,0.733800,0.745441
200,0.743000,0.717768
300,0.690700,0.707936
400,0.705300,0.699224
500,0.636000,0.699367
600,0.645500,0.696530
700,0.640700,0.695393


Unsloth: Not an error, but Qwen2ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


✅ Training completed!
Training loss: 0.7439


### Thử lại sau khi finetune

In [9]:
# ===========================
# 6. Lưu model
# ===========================

# Lưu LoRA adapters
model.save_pretrained("absa_lora_model")
tokenizer.save_pretrained("absa_lora_model")

print("✅ Model saved to 'absa_lora_model/'")

# Merge LoRA và lưu full model (optional)
model.save_pretrained_merged("absa_full_model", tokenizer, save_method="merged_16bit")
print("✅ Merged model saved to 'absa_full_model/'")



✅ Model saved to 'absa_lora_model/'


config.json:   0%|          | 0.00/757 [00:00<?, ?B/s]

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Checking cache directory for required files...
Cache check failed: model-00001-of-00002.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


...

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Unsloth: Preparing safetensor model files: 100%|██████████| 2/2 [00:25<00:00, 12.60s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 2/2 [00:44<00:00, 22.14s/it]


Unsloth: Merge process complete. Saved to `/kaggle/working/absa_full_model`
✅ Merged model saved to 'absa_full_model/'


In [10]:
# ===========================
# 7. Test model sau khi train
# ===========================

FastLanguageModel.for_inference(model)

test_comment = "Pin trâu, màn hình đẹp nhưng camera hơi mờ"
test_messages = [
    {"role": "system", "content": """Bạn là chuyên gia phân tích đánh giá sản phẩm điện thoại.
Nhiệm vụ: Trích xuất các khía cạnh (aspect) và phân loại cảm xúc (sentiment) từ đánh giá.

Các khía cạnh hợp lệ: SCREEN, CAMERA, FEATURES, BATTERY, PERFORMANCE, STORAGE, DESIGN, PRICE, GENERAL, SER&ACC
Các cảm xúc hợp lệ: Positive, Negative, Neutral"""},
    {"role": "user", "content": f'Phân tích đánh giá: "{test_comment}"'}
]

inputs = tokenizer.apply_chat_template(
    test_messages,
    add_generation_prompt=True,
    return_tensors="pt",
    return_dict=True,
).to(model.device)

from transformers import TextStreamer
print("\n🧪 Testing model:")
print(f"Input: {test_comment}")
print("Output:")

outputs = model.generate(
    **inputs,
    max_new_tokens=256,
    streamer=TextStreamer(tokenizer),
    temperature=0.1,
    top_p=0.9,
)

print("\n✅ Test completed!")


🧪 Testing model:
Input: Pin trâu, màn hình đẹp nhưng camera hơi mờ
Output:
<|im_start|>system
Bạn là chuyên gia phân tích đánh giá sản phẩm điện thoại.
Nhiệm vụ: Trích xuất các khía cạnh (aspect) và phân loại cảm xúc (sentiment) từ đánh giá.

Các khía cạnh hợp lệ: SCREEN, CAMERA, FEATURES, BATTERY, PERFORMANCE, STORAGE, DESIGN, PRICE, GENERAL, SER&ACC
Các cảm xúc hợp lệ: Positive, Negative, Neutral<|im_end|>
<|im_start|>user
Phân tích đánh giá: "Pin trâu, màn hình đẹp nhưng camera hơi mờ"<|im_end|>
<|im_start|>assistant
ASPECT: SCREEN, CAMERA, BATTERY
SENTIMENT: Positive<|im_end|>

✅ Test completed!
